<a href="https://colab.research.google.com/github/praneshnikhar/Deep-Learning-models/blob/main/ML_language_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM ,Dense
import numpy as np

batch_size =64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = '/content/eng-fra.txt'

In [4]:
input_texts=[]
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding = 'utf-8') as f:
  lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines)-1)]:
  input_text, target_text,  = line.split('\t')

  target_text = '\t'+target_text +'\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [5]:
target_texts

['\tVa !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tÇa alors\u202f!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tJe comprends.\n',
 "\tJ'essaye.\n",
 "\tJ'ai gagné !\n",
 "\tJe l'ai emporté !\n",
 '\tOh non !\n',
 '\tAttaque !\n',
 '\tAttaquez !\n',
 '\tSanté !\n',
 '\tÀ votre santé !\n',
 '\tMerci !\n',
 '\tLève-toi.\n',
 "\tJ'ai pigé !\n",
 '\tCompris !\n',
 '\tPigé\u202f?\n',
 '\tCompris\u202f?\n',
 "\tT'as capté\u202f?\n",
 '\tMonte.\n',
 '\tMontez.\n',
 '\tSerre-moi dans tes bras !\n',
 '\tSerrez-moi dans vos bras !\n',
 '\tJe suis tombée.\n',
 '\tJe suis tombé.\n',
 '\tJe sais.\n',
 '\tJe suis parti.\n',
 '\tJe suis partie.\n',
 "\tJ'ai perdu.\n",
 "\tJ'ai 19 ans.\n",
 '\tJe vais bien.\n',
 '\tÇa va.\n',
 '\tÉcoutez !\n',
 '\tImpossible\u202f!\n',
 '\tEn aucun cas.\n',
 "\tC'est hors de question !\n",
 "\tIl n'en est pas question !\n",
 "\tC'est exclu 

In [6]:
target_characters

{'\t',
 '\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '5',
 '6',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '‘',
 '’',
 '\u202f'}

In [7]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [8]:
print('Number of samples: ', len(input_texts))
print('Number of unique input tokens: ', num_encoder_tokens)
print('Number of unique output tokens: ', num_decoder_tokens)
print('Max sequence length for inputs: ', max_encoder_seq_length)
print('Max sequence length for outputs: ', max_decoder_seq_length)

Number of samples:  10000
Number of unique input tokens:  71
Number of unique output tokens:  91
Max sequence length for inputs:  16
Max sequence length for outputs:  59


In [9]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [11]:
input_token_index

{' ': 0,
 '!': 1,
 '$': 2,
 '&': 3,
 "'": 4,
 ',': 5,
 '-': 6,
 '.': 7,
 '0': 8,
 '1': 9,
 '2': 10,
 '3': 11,
 '4': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '9': 16,
 ':': 17,
 '?': 18,
 'A': 19,
 'B': 20,
 'C': 21,
 'D': 22,
 'E': 23,
 'F': 24,
 'G': 25,
 'H': 26,
 'I': 27,
 'J': 28,
 'K': 29,
 'L': 30,
 'M': 31,
 'N': 32,
 'O': 33,
 'P': 34,
 'Q': 35,
 'R': 36,
 'S': 37,
 'T': 38,
 'U': 39,
 'V': 40,
 'W': 41,
 'Y': 42,
 'Z': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 '’': 70}

In [13]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '$': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '5': 12,
 '6': 13,
 '9': 14,
 ':': 15,
 '?': 16,
 'A': 17,
 'B': 18,
 'C': 19,
 'D': 20,
 'E': 21,
 'F': 22,
 'G': 23,
 'H': 24,
 'I': 25,
 'J': 26,
 'K': 27,
 'L': 28,
 'M': 29,
 'N': 30,
 'O': 31,
 'P': 32,
 'Q': 33,
 'R': 34,
 'S': 35,
 'T': 36,
 'U': 37,
 'V': 38,
 'Y': 39,
 'Z': 40,
 'a': 41,
 'b': 42,
 'c': 43,
 'd': 44,
 'e': 45,
 'f': 46,
 'g': 47,
 'h': 48,
 'i': 49,
 'j': 50,
 'k': 51,
 'l': 52,
 'm': 53,
 'n': 54,
 'o': 55,
 'p': 56,
 'q': 57,
 'r': 58,
 's': 59,
 't': 60,
 'u': 61,
 'v': 62,
 'w': 63,
 'x': 64,
 'y': 65,
 'z': 66,
 '\xa0': 67,
 '«': 68,
 '»': 69,
 'À': 70,
 'Ç': 71,
 'É': 72,
 'Ê': 73,
 'à': 74,
 'â': 75,
 'ç': 76,
 'è': 77,
 'é': 78,
 'ê': 79,
 'ë': 80,
 'î': 81,
 'ï': 82,
 'ô': 83,
 'ù': 84,
 'û': 85,
 'œ': 86,
 '\u2009': 87,
 '‘': 88,
 '’': 89,
 '\u202f': 90}

In [18]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype = 'float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype = 'float32')

In [19]:
for i ,(input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  encoder_input_data[i, t+1:, input_token_index[' ']] = 1.
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t>0:
      decoder_target_data[i, t-1, target_token_index[char]] = 1.
  decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
  decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [20]:
encoder_input_data[0].shape

(16, 71)

In [21]:
encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [22]:
decoder_inputs = Input(shape = (None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [23]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 57s 433ms/step - accuracy: 0.6837 - loss: 1.6418 - val_accuracy: 0.6891 - val_loss: 1.2372
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 78s 403ms/step - accuracy: 0.7263 - loss: 1.0278 - val_accuracy: 0.6891 - val_loss: 1.2497
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 50s 398ms/step - accuracy: 0.7424 - loss: 0.9300 - val_accuracy: 0.7323 - val_loss: 0.9979
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 91s 473ms/step - accuracy: 0.7677 - loss: 0.8383 - val_accuracy: 0.7481 - val_loss: 0.8910
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 58s 465ms/step - accuracy: 0.7898 - loss: 0.7331 - val_accuracy: 0.7627 - val_loss: 0.8185
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 54s 432ms/step - accuracy: 0.8029 - loss: 0.6828 - val_accuracy: 0.7762 - val_loss: 0.7728
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 50s 404ms/step - accuracy: 0.8123 - loss: 0.6431 - val_accuracy: 0.7776 - val_loss: 0.7581
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 51s 411ms/step - accuracy: 0.8172 -

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = input(shape = (latent_dim,))
decoder_state_input_c = input(shape = (latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs]+ decoder_states
                      [decoder_outputs]+decoder_states)

